In [ ]:
import config as cfg
import logging

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
METADATA_LJY_PATH = "../_data/all_data_with_severity.csv"

In [ ]:
demo_cols = [
    'gender', 'age', 'marriage', 'job', 'smkHx', 'drinkHx', 'suicideHx'
]
dailylog_cols = [
    'panic', 'severity', 'exercise', 'alcohol', 'coffee', 'menstruation',
	'smoking', 'positive_feeling', 'negative', 'positive_E', 'negative_E',
	'anxiety', 'annoying'
]
lifelog_cols = [
	'HR_var', 'HR_max', 'HR_mean', 'HR_hvar_mean', 'HR_acrophase', 'HR_amplitude',
 	'HR_mesor','HR_acrophase_difference', 'HR_acrophase_difference_2d', 'HR_amplitude_difference',
  	'HR_amplitude_difference_2d', 'HR_mesor_difference', 'HR_mesor_difference_2d',
	'bandpower(0.001-0.0005Hz)', 'bandpower(0.0005-0.0001Hz)', 'bandpower(0.0001-0.00005Hz)', 'bandpower(0.00005-0.00001Hz)',
	'steps', 'SLT1', 'SLT2', 'SLT3', 'SLT4', 'SLT5', 'SLT6', 'total_sleep'
]
questionnaire_cols = [
	'PHQ_9', 'STAI_X2', 'CSM', 'CTQ_1', 'CTQ_2', 'CTQ_3', 'CTQ_4', 'CTQ_5', 'KRQ', 'MDQ',
 	'ACQ', 'APPQ_1', 'APPQ_2', 'APPQ_3', 'BSQ', 'GAD_7', 'BRIAN'
]
all_cols = demo_cols + dailylog_cols + lifelog_cols + questionnaire_cols
print(f'Number of columns: {len(all_cols)}')

In [ ]:
metadata_ljy = pd.read_csv(METADATA_LJY_PATH)

# check if all columns are present
missing_cols = [col for col in all_cols if col not in metadata_ljy.columns]
if missing_cols:
    print(f"Missing columns in metadata_ljy: {missing_cols}")
# convert date column to datetime format
metadata_ljy['date'] = pd.to_datetime(metadata_ljy['date'], format='%Y-%m-%d')

In [ ]:
print(f"Number of rows: {metadata_ljy.shape[0]}")
print(f"Number of columns: {metadata_ljy.shape[1]}")
display(metadata_ljy.head(5))

In [ ]:
# Add 'dataset' column: source of data
metadata_ljy['dataset'] = metadata_ljy['ID'].str.split('_').str[0]
metadata_ljy['dataset'] = metadata_ljy['dataset'].str.split('-').str[0]
unique_dataset = metadata_ljy['dataset'].unique()
print("Unique sources in metadata_ljy:")

print(unique_dataset)
print("--------------------------------------")
print("Total number of entries:", metadata_ljy.shape[0])
pxpn_n = metadata_ljy[metadata_ljy['dataset'] == 'PXPN'].shape[0]
print("    PXPN dataset:", pxpn_n)
sym1_n = metadata_ljy[metadata_ljy['dataset'] == 'SYM1'].shape[0]
sym2_n = metadata_ljy[metadata_ljy['dataset'] == 'SYM2'].shape[0]
print("    SYM dataset:", sym1_n+sym2_n)
print("--------------------------------------")
unique_ids = metadata_ljy['ID'].unique()
print("Number of unique IDs in metadata_ljy:", len(unique_ids))
n_panic_2 = metadata_ljy[metadata_ljy['panic'] == 2].shape[0]
print("Number of panic events (panic=2):", n_panic_2)

display(metadata_ljy.head(5))

In [ ]:
from library.pandas_utils import aggregate_by_column

agg_matrix = [
	('n_entries', 'ID', 'count'),
	('n_dates', 'date', 'nunique'),
	('n_panic_2', 'panic', lambda x: (x == 2).sum())
]

metadata_ljy_agg = aggregate_by_column(metadata_ljy, 'ID', agg_matrix)
#display(metadata_ljy_agg.head(5))

In [ ]:
def plot_histogram_of_counts(column, title=None, xlabel=None, ylabel="Frequency", bins_step=5):
	"""
	Plot a histogram of counts for a given pandas Series (column).
	Only nonzero values are plotted.
	"""
	import matplotlib.pyplot as plt

	data = column[column > 0]
	if data.empty:
		print("No nonzero values to plot.")
		return

	min_val = int(data.min())
	max_val = int(data.max())
	bins = np.arange(min_val, max_val + 2)  # +2 to include max in bin edges

	plt.figure(figsize=(10, 4))
	plt.hist(data, bins=bins, color='blue', alpha=0.7)
	plt.title(title or f'Histogram of {column.name}')
	plt.xlabel(xlabel or column.name)
	plt.ylabel(ylabel)
	plt.grid(axis='y', alpha=0.75)
	plt.xticks(np.arange(0, max_val + 1, bins_step))
	plt.tight_layout()
	plt.show()

In [ ]:
# Find all IDs that ever had panic==2
panic_ids = metadata_ljy_agg.loc[
    metadata_ljy_agg['n_panic_2'] > 0, 'ID'
].unique()
print("Unique IDs with panic events (panic=2):", len(panic_ids))
print(f"Number of panic events (panic=2): {n_panic_2}")
print("--------------------------------------")
plot_histogram_of_counts(metadata_ljy_agg['n_panic_2'], title="Histogram of Panic Events per ID", xlabel="Number of Panic Events")

In [ ]:
delta_days = 3

# Make a fresh copy of metadata_ljy and add four new columns
metadata = metadata_ljy.copy()
metadata['last_panic_days'] = None
metadata['n_prior_data']    = None
metadata['event_id']        = None
metadata['ref_event_id']    = None

# Loop over each subject‐ID that has at least one panic==2
for panic_id in panic_ids:
    # Extract only rows for this subject, sorted by date ascending
    df = metadata.loc[metadata['ID'] == panic_id].sort_values('date', ascending=True)

    # Build a set of all observed dates for quick membership tests
    date_set = set(df['date'])

    last_panic_date = None

    for idx, row in df.iterrows():
        if row['panic'] != 2:
            # Only process rows where panic==2
            continue

        # Create and store event_id
        event_id = f"{panic_id}_{row['date'].date()}"
        metadata.loc[idx, 'event_id'] = event_id

        # Compute last_panic_days
        if last_panic_date is None:
            # This is the very first panic for this subject
            metadata.loc[idx, 'last_panic_days'] = 100
        else:
            days_diff = (row['date'] - last_panic_date).days
            metadata.loc[idx, 'last_panic_days'] = days_diff

        # Count how many consecutive prior days (up to 100) exist before this panic
        # We look backward one day at a time. If we hit another panic, or a missing date, we break.
        current_date = row['date']
        found_n_prior = False

        for j in range(1, 101):  # j = 1, 2, …, 100
            look_date = current_date - pd.Timedelta(days=j)

            if look_date not in date_set:
                # We encountered a missing day. That means only (j-1) consecutive prior days exist.
                metadata.loc[idx, 'n_prior_data'] = j - 1
                found_n_prior = True
                break

            # There *is* at least one row on look_date—grab it (or them)
            rows_on_that_date = df[df['date'] == look_date]
            if len(rows_on_that_date) > 1:
                print(f"Warning: More than one row for date {look_date} for ID {panic_id}")
            # If ANY of those rows had panic==2, we stop and record (j-1):
            if (rows_on_that_date['panic'] == 2).any():
                metadata.loc[idx, 'n_prior_data'] = j - 1
                found_n_prior = True
                break

            if j in range(1, delta_days + 1):
                # Set the 'ref_event_id' to event_id for all the rows for rows_on_that_date if we are within the delta_days
                metadata.loc[rows_on_that_date.index, 'ref_event_id'] = event_id
  
        if not found_n_prior:
            # We never hit a prior panic or missing day within 100 days → cap at 100
            metadata.loc[idx, 'n_prior_data'] = 100

        # update last_panic_date so that the next panic (if any) calculates correctly
        last_panic_date = row['date']

In [ ]:
agg_matrix = [
	('n_entries', 'ref_event_id', 'count'),
	('n_dates', 'date', 'nunique')
]

metadata_agg = aggregate_by_column(metadata, 'ref_event_id', agg_matrix)
#display(metadata_agg.head(5))

check = metadata_agg[metadata_agg['n_entries'] != metadata_agg['n_dates']]
#print("Entries where n_entries != n_dates:")
#display(check)

In [ ]:
# Filter down to only those rows we marked as panic events
panic_data = metadata[['ID', 'date', 'event_id', 'last_panic_days', 'n_prior_data', 'severity']].copy()
panic_data = panic_data[panic_data['event_id'].notnull()]

panic_data = panic_data[
    (panic_data['last_panic_days'] > delta_days) &
    (panic_data['n_prior_data'] >= delta_days) &
    (panic_data['severity'].notnull())
]
print(f"-------- last_panic_days > {delta_days} & n_prior_data ≥ {delta_days} --------")
print(f"Number of qualifying panic events: {panic_data.shape[0]} out of {n_panic_2} ({panic_data.shape[0] / n_panic_2:.2%})")
print(f"Unique IDs with panic events: {len(panic_data['ID'].unique())} out of {len(panic_ids)} ({len(panic_data['ID'].unique()) / len(panic_ids):.2%})")

display(panic_data.head(5))

In [ ]:
filtered_metadata = metadata[(metadata['ref_event_id'].notnull()) | (metadata['event_id'].notnull())].copy()
qulifying_event_ids = panic_data['event_id'].unique()
qualifying_metadata = filtered_metadata[filtered_metadata['ref_event_id'].isin(qulifying_event_ids)]
display(qualifying_metadata.head(10))
print(f"Expected number of columns: {len(all_cols) * 3}")

disp_data = filtered_metadata[['ID', 'date', 'event_id', 'ref_event_id', 'last_panic_days', 'n_prior_data']].copy()
display(disp_data.head(50))
del disp_data

In [ ]:
agg_matrix = [
	('n_entries', 'ref_event_id', 'count'),
	('n_dates', 'date', 'nunique')
]

qualifying_metadata_agg = aggregate_by_column(qualifying_metadata, 'ref_event_id', agg_matrix)
#display(filtered_metadata_agg.head(5))
plot_histogram_of_counts(qualifying_metadata_agg['n_entries'], title="Histogram of Entries per Ref Event ID", xlabel="Number of Entries", bins_step=1)
check = qualifying_metadata_agg[qualifying_metadata_agg['n_entries'] != qualifying_metadata_agg['n_dates']]
print("Entries where n_entries != n_dates:")
display(check)

In [ ]:
unique_panic = metadata_ljy['severity'].unique()
print(f"\nUnique values in 'panic': {unique_panic}")